In [1]:
trainSize = "2000examples_400x400complex.mat"  #Other datasets ['dataShort.mat', 'data1000.mat', 'dataFull.mat']

model_name = 'Cophy-PGNN/pretrained/CoPhy.pt'

In [2]:
# parameters
depth=2
hidden=100
out = 804
in_ = 10

In [3]:
import time
import torch
import random
import numpy as np
import pandas as pd
import matplotlib

import sys
sys.path.append('../../scripts')
sys.path.append('../')

from util.solvers import *
from util.utils import eval_all
from data_loader_electromagnetic import DatasetLoader

sys.path.append('Cophy-PGNN/')
from DNN import DNN

In [4]:
torch.cuda.set_device(6)
device = torch.device('cuda:6')

In [5]:
class Electromagnetic_dataset(object):
    def __init__(self):
        data = DatasetLoader(
            train_size=trainSize, #,
            data_path='//home/elhamod/CoPhy-PGNN/CoPhy-PGNN/datasets/electromagnetic_dualphase', 
        )
        
        # normalization
        data.normalization(x=False, y=False)
        
        self.input_=data.X_test
        H=data.H_test_origin
        H_complex_width = int(H.shape[1]/2)
        
        H_real = H[:, :H_complex_width, :H_complex_width]
        H_img = H[:, H_complex_width:, :H_complex_width]
        H_complex = torch.stack([H_real, H_img]).permute(1, 2, 3, 0).contiguous().numpy()
        self.H = H_complex[:, :, :, 0] + 1j*H_complex[:, :, :, 1]

        self.data = data
        self.cnt=0
       
    def count(self):
        return self.H.shape[0]
        
    def get(self):
        cnt = self.cnt
        self.cnt = self.cnt+1
        return self.input_[cnt, :], self.H[cnt,:,:]
    


# ======================= PyTorch ========================  


class CophyPGNN(EigenSolver):
    def __init__(self):
        super().__init__()
        
        with torch.no_grad(): 
            model_gpu = DNN(in_, hidden, out, depth, act=torch.nn.Tanh)
            model_gpu.load_state_dict(torch.load(model_name)) 
            model_gpu = model_gpu.double().cuda()
            model_gpu.eval()
            
        self.solvers = {
            'cophy-pgnn_cuda': model_gpu,
        }
    
    def run(self, m, sort=True):
        m,h = m

        results = self.init_dict()
        runtime = self.init_dict()
        errors = self.init_dict()
        
        for name in self.solvers:
            solver = self.solvers[name]
            elapse = 0

                        
            if 'cuda' in name:
                device = torch.device('cuda')
            else:
                device = torch.device('cpu')
            
            
            # solve and record runtime
            m = torch.tensor(m, dtype=torch.double).cuda(device)
            start = time.process_time()             # start
            with torch.no_grad():
                result = solver(m).detach().cpu().numpy()
            elapse += time.process_time() - start   # end
        
            val = result[-2:]
            vec = result[:-2]

            # post-exec sort results
            val = val[0] + 1j*val[1]
            
            V_complex_width = int(vec.shape[0]/2)
            vec = vec[:V_complex_width] + 1j*vec[V_complex_width:]
            
            # record
            runtime[name] = elapse
            results[name] = (val, vec)
            errors[name] = np.mean(np.abs(h @ vec - val * vec))
            n = np.linalg.norm(vec)
            
        return runtime, n, errors
    
    


In [6]:
dataset = Electromagnetic_dataset()

../../scripts/data_loader_electromagnetic.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_train_origin = torch.tensor(self.y_train).float()
../../scripts/data_loader_electromagnetic.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_val_origin = torch.tensor(self.y_val).float()
../../scripts/data_loader_electromagnetic.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_test_origin = torch.tensor(self.y_test).float()


In [7]:
solvers = [CophyPGNN(), NumpySolvers(), TorchSolvers(), ScipySolvers(), ScipySparse()] # 
df , norms= eval_all(dataset, solvers)


df.to_csv("results/stats.csv")


100%|██████████| 1629/1629 [39:00<00:00,  1.44s/it]


In [8]:
df

,avg time,avg L1 error
cophy-pgnn_cuda,0.037299,1.878558e+02
numpy_eig,31.221107,7.714403e-06
torch_cuda,13.891261,6.821139e-13
scipy_eig,29.998622,7.538649e-04
scipy_sparse_eig,0.956335,4.418413e-03
